In [11]:
from langchain_community.graphs import Neo4jGraph
from neo4j.debug import watch
import pandas as pd
from pyprojroot import here
watch("neo4j")

In [12]:
NEO4J_URL = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "12345678"
NEO4J_DATABASE = 'neo4j'

In [15]:
graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,355  [#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,355  [#0000]  _: <POOL> created, direct address IPv4Address(('localhost', 7687))
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,357  [#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=None
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,357  [#0000]  _: <POOL> acquire direct connection, access_mode='READ', database=None
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,357  [#0000]  _: <POOL> trying to hand out new connection
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40,357  [#0000]  _: <POOL> trying to hand out new connection
[DEBUG   ] [Thread 140533884981248] [Task 140533814188400] 2024-04-27 20:47:40

ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

**Nodes:**
- `Movie`: Represents a movie. Each movie node has <u>attributes</u> such as **id** (a unique identifier for the movie), **released** (the release date of the movie), **title** (the movie's title), and **imdbRating** (the movie's rating on IMDb).
- `Person`: Represents an individual who can either be an <u>actor</u> or a <u>director</u> (or both) in movies. Each person node has a <u>single attribute</u>, **name**, which is the name of the person.
- `Genre`: Represents a movie genre. Each genre node has a <u>single attribute</u>, **name**, which is the genre type (e.g., Action, Comedy, Drama, etc.).

**Relationships:**
- `:DIRECTED`: A directional relationship from a Person node to a Movie node, signifying that the person directed the movie.
- `:ACTED_IN`: A directional relationship from a Person node to a Movie node, signifying that the person acted in the movie.
- `:IN_GENRE`: A directional relationship from a Movie node to a Genre node, signifying that the movie belongs to that particular genre.

**Instructions in the script:**
- `LOAD CSV WITH HEADERS`: Loads a CSV file that contains the movie data with headers indicating each column's purpose.
- `MERGE`: Ensures that a node or relationship is created if it does not already exist; otherwise, it matches the existing node or relationship. This prevents duplication.
- `SET`: Assigns properties to the nodes after they've been created or matched.
- `FOREACH`: Executes the contained commands for each element in a list. This is used to iterate over the lists of directors, actors, and genres associated with each movie. It ensures that all the respective Person and Genre nodes are created and linked appropriately to the Movie nodes.

**The Graph Structure:**
- Each Movie node is connected to one or more Person nodes by either :DIRECTED or :ACTED_IN relationships, depending on whether the person is listed as a director or an actor of that movie.
- Each Movie node is also connected to one or more Genre nodes by the :IN_GENRE relationship, representing the genres that the movie is categorized under.

This script effectively takes movie data from a CSV and constructs a rich graph that interlinks movies with the people who directed and acted in them, as well as the genres to which they belong. It is a typical graph structure for a recommendation system or for analysis of relationships within the movie industry.

**To load the data withing cypher query:**

--------------------
- NOTE: Uncomment `dbms.security.allow_csv_import_from_file_urls=true` in `neo4j.conf` to be able to load the file locally.
- NOTE: Absolute path + no space for the file directory.
--------------------

In [6]:
movie_csv_path = here("data/movie_csv/movie.csv")
movie_csv_path

WindowsPath('c:/Users/froozitalab/OneDrive - R.W. Tomlinson Ltd/Documents/Codes/LLM-Zero-to-Hundred/KnowledgeGraph-RAG-GPT/data/movie_csv/movie.csv')

In [7]:
print(pd.read_csv('c:/Users/froozitalab/OneDrive - R.W. Tomlinson Ltd/Documents/Codes/LLM-Zero-to-Hundred/KnowledgeGraph-RAG-GPT/data/movie_csv/movie.csv').columns)

Index(['Unnamed: 0', 'movieId', 'released', 'title', 'actors', 'director',
       'genres', 'imdbRating', 'tagline', 'taglineEmbedding'],
      dtype='object')


In [8]:
print(pd.read_csv('c:/Users/froozitalab/movie.csv').columns)

Index(['movieId', 'released', 'title', 'actors', 'director', 'genres',
       'imdbRating', 'tagline'],
      dtype='object')


In [10]:
# import movie information from the CSV file with tagline
movies_query = """
    LOAD CSV WITH HEADERS FROM 'file:///c:/Users/froozitalab/movie.csv' 
    AS row
    MERGE (m:Movie {id:row.movieId})
    SET m.released = date(row.released),
        m.tagline = row.tagline,
        m.title = row.title,
        m.imdbRating = toFloat(row.imdbRating),
    FOREACH (director in split(row.director, '|') | 
        MERGE (p:Person {name:trim(director)})
        MERGE (p)-[:DIRECTED]->(m))
    FOREACH (actor in split(row.actors, '|') | 
        MERGE (p:Person {name:trim(actor)})
        MERGE (p)-[:ACTED_IN]->(m))
    FOREACH (genre in split(row.genres, '|') | 
        MERGE (g:Genre {name:trim(genre)})
        MERGE (m)-[:IN_GENRE]->(g))
"""

In [11]:
graph.query(movies_query)

[]

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Movie {tagline: STRING, title: STRING, imdbRating: FLOAT, id: STRING, released: DATE},Person {name: STRING},Genre {name: STRING}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


In [13]:
# Match all nodes in the graph
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = graph.query(cypher)
result

[{'count(n)': 132}]

### **Create vector embedding from the `tagline` column**

In [1]:
import os
import pandas as pd

In [11]:
movie_csv_path = here("data/movie_csv/movie.csv")
df = pd.read_csv(movie_csv_path)
df.columns

Index(['Unnamed: 0', 'movieId', 'released', 'title', 'actors', 'director',
       'genres', 'imdbRating', 'tagline', 'taglineEmbedding'],
      dtype='object')

In [12]:
model_name = "gpt-35-turbo-1106"
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

**Load the Azure OpenAI Embedding Model**

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = azure_openai_api_key,  
  api_version = "2023-07-01-preview",
  azure_endpoint = azure_openai_endpoint
)
def embed_text(text):
    response = client.embeddings.create(
    input = text,
    model= "text-embedding-ada-002"
    )
    return response.data[0].embedding

In [ ]:
embedding_list = [embed_text(i) for i in df["tagline"]]

In [ ]:
df["taglineEmbedding"] = embedding_list

**Create a vector index**

In [ ]:
graph.query("""
  CREATE VECTOR INDEX movie_tagline_embeddings IF NOT EXISTS
  FOR (m:Movie) ON (m.taglineEmbedding) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 1536,
    `vector.similarity_function`: 'cosine'
  }}"""
)

In [ ]:
graph.query("""
  SHOW VECTOR INDEXES
  """
)

**Populate the index**

In [ ]:
for index, row in df.iterrows():
    movie_id = row['movieId']
    embedding = row['taglineEmbedding']
    graph.query(f"MATCH (m:Movie {{id: '{movie_id}'}}) SET m.taglineEmbedding = {embedding}")

**Verify that the index was created successfuly**

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
result = graph.query("""
    MATCH (m:Movie) 
    WHERE m.tagline IS NOT NULL
    RETURN m.tagline, m.taglineEmbedding
    LIMIT 1
    """
)

In [ ]:
result[0]['m.tagline']

In [ ]:
result[0]['m.taglineEmbedding']